# C4021 - Group Project 2.3

### Problem Statement: Telecom Customer Churn Prediction

Classification Problem to predict the Telecom customer churn which is one of the major problem Telecom industry is facing today.


## Dataset Information

####   Source: https://www.kaggle.com/blastchar/telco-customer-churn 

#### Context: 
"Predict behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs." [IBM Sample Data Sets]

#### Content: 
Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

##### The data set includes information about:
- Customers who left within the last month – the column is called Churn
- Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
- Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
- Demographic info about customers – gender, age range, and if they have partners and dependents

## Team Contributions:

#### Training data
#### Pre-processing
#### Algorithm training and evaluation
#### Visualization of outputs